## Using positive samples from previous years


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2

Loading data from 2020 competition:

In [2]:
train_df = pd.read_csv("train.csv")

In [3]:
def append_ext(filename):
    return filename + ".png"

In [4]:
train_df['image_name'] = train_df['image_name'].apply(append_ext)

Next, we load data from previous years:

In [5]:
prev_df = pd.read_csv(r'D:\uni\masters\ML\project\my_code\old_data\ISIC_2019_Training_GroundTruth.csv')

In [6]:
prev_df.head()

,image,MEL,NV,BCC,AK,BKL,DF,VASC,SCC,UNK
0,ISIC_0000000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ISIC_0000001,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,ISIC_0000002,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,ISIC_0000003,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,ISIC_0000004,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Next, we extract melanoma-positive samples:

In [7]:
prev_df.shape[0]

25331

Extracting only positive samples:

In [8]:
prev_positive = prev_df[prev_df['MEL'] == 1]

In [9]:
prev_positive.shape[0]

4522

In [10]:
import os

In [11]:
old_positive = pd.read_csv("old_positive.csv")

In [12]:
old_positive.head()

,Unnamed: 0,image_name,target
0,0,oldISIC_0000002,1
1,1,oldISIC_0000004,1
2,2,oldISIC_0000013,1
3,3,oldISIC_0000022_downsampled,1
4,4,oldISIC_0000026_downsampled,1


In [13]:
old_positive.shape[0]

4522

In [14]:
old_positive['image_name'] = old_positive['image_name'].apply(append_ext)

In [15]:
old_positive.head()

,Unnamed: 0,image_name,target
0,0,oldISIC_0000002.png,1
1,1,oldISIC_0000004.png,1
2,2,oldISIC_0000013.png,1
3,3,oldISIC_0000022_downsampled.png,1
4,4,oldISIC_0000026_downsampled.png,1


In [16]:
train = train_df[['image_name', 'target']]

How many positive samples are present in the training data set:

In [17]:
train[train['target'] == 1].shape[0]

584

In [18]:
n = train[train['target'] == 1].shape[0] + old_positive.shape[0]

In [19]:
print(f"total number of positive samples: {n}")

total number of positive samples: 5106


In [20]:
train_positive = train[train['target'] == 1]

In [21]:
train_positive.shape[0]

584

We downsample negative samples:

In [22]:
train_negative = train[train['target'] == 0].sample(3*n, 
                                                   random_state=111)

In [23]:
train_negative.shape[0]

15318

In [24]:
train_negative[train_negative['target'] == 1]

,image_name,target


In [25]:
train = pd.concat([old_positive, train_positive])

In [26]:
train.shape[0]

5106

In [27]:
train = pd.concat([train, train_negative])

In [28]:
train.shape[0]

20424

In [29]:
train.to_csv('downsampled_plus_old.csv')

Validation set demontrated to be ineffective. This time we will try to use all the data for training.

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [31]:
train_datagen = ImageDataGenerator(rotation_range=90,
                                   rescale=1./255,
                                   shear_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   channel_shift_range=0.7,
                                   brightness_range=[0.7,1.3],
                                   zoom_range=[0.8, 1.2],
                                   horizontal_flip=True,
                                   #fill_mode='nearest',
                                   vertical_flip=True)
val_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train,
    directory = r'D:\uni\masters\ML\project\my_code\combined_data_set',
    x_col = 'image_name',
    y_col = 'target',
    batch_size = 32,
    target_size=(224, 224),
    shuffle = True,
    class_mode = 'raw',
    seed = 111
)



Found 20424 validated image filenames.


## GPU settings

In [32]:
import tensorflow as tf

In [33]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


In [34]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
import keras
import PIL
from tqdm.keras import TqdmCallback

Using TensorFlow backend.


## Loading pre-trained InceptionV3


In [35]:
input_tensor = Input(shape=(224, 224, 3))

In [36]:
inceptionv3_model = tf.keras.applications.InceptionV3(weights='imagenet',
                                                     input_tensor=input_tensor,
                                                     include_top=False)


Adding new layers on top of InceptionV3:

In [37]:
x = inceptionv3_model.output

In [38]:
x = GlobalAveragePooling2D()(x)

In [39]:
x = Dense(units=1024, activation='relu')(x)

In [40]:
x = Dropout(0.5)(x)

In [42]:
x = Dense(units=512, activation='relu')(x)

In [43]:
x = Dropout(0.3)(x)

In [44]:
predictions = Dense(units=1, activation='sigmoid')(x)

In [45]:
model = Model(inputs=inceptionv3_model.input,
             outputs=predictions)

First, we train newly added layers:

In [46]:
for layer in inceptionv3_model.layers:
    layer.trainable = False

In [47]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*batch_size)

In [48]:
model.compile(optimizer='rmsprop',
             loss='binary_crossentropy',
             metrics=['AUC'])

In [49]:
# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

In [50]:
history = model.fit(
    train_generator,
    epochs = 20,
    verbose=0, 
    callbacks=[TqdmCallback(verbose=2), cp_callback]
)

  ...
    to  
  ['...']



Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model t


Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model t


Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model t


Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model t

...


Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model to training_2/cp-0016.ckpt

Epoch 00016: saving model t


Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model to training_2/cp-0017.ckpt

Epoch 00017: saving model t


Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model to training_2/cp-0018.ckpt

Epoch 00018: saving model t


Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model to training_2/cp-0019.ckpt

Epoch 00019: saving model t


Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00020: saving model t

In [47]:
ls {checkpoint_dir}

 Volume in drive D is Ќ®ўл© в®¬
 Volume Serial Number is F6A5-E931

 Directory of D:\uni\masters\ML\project\my_code\training_2

12/10/2020  09:50 AM    <DIR>          .
12/10/2020  09:50 AM    <DIR>          ..
12/10/2020  09:50 AM                81 checkpoint
12/10/2020  07:26 AM            60,164 cp-0000.ckpt.data-00000-of-00002
12/10/2020  07:26 AM        95,607,940 cp-0000.ckpt.data-00001-of-00002
12/10/2020  07:26 AM            25,532 cp-0000.ckpt.index
12/10/2020  07:31 AM            61,234 cp-0001.ckpt.data-00000-of-00002
12/10/2020  07:31 AM       104,004,768 cp-0001.ckpt.data-00001-of-00002
12/10/2020  07:31 AM            26,189 cp-0001.ckpt.index
12/10/2020  07:34 AM            61,234 cp-0002.ckpt.data-00000-of-00002
12/10/2020  07:34 AM       104,004,768 cp-0002.ckpt.data-00001-of-00002
12/10/2020  07:34 AM            26,189 cp-0002.ckpt.index
12/10/2020  07:38 AM            61,234 cp-0003.ckpt.data-00000-of-00002
12/10/2020  07:38 AM       104,004,768 cp-0003.ckpt.data-0000

In [51]:
test_df = pd.read_csv('test.csv')

In [52]:
i = 0
predictions = []
for image_name in test_df['image_name']:
    i += 1
    print(f"image {i}/{test_df.shape[0]}")
    filename = image_name+".png"
    img = cv2.imread(r'D:\uni\masters\ML\project\my_code\archive\test\\'+filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = tf.keras.applications.inception_v3.preprocess_input(img)
    x = np.reshape(x, (1, 224, 224, 3))
    target = model.predict(x)
    predictions.append(target[0][0])

image 1/10982
image 2/10982
image 3/10982
image 4/10982
image 5/10982
image 6/10982
image 7/10982
image 8/10982
image 9/10982
image 10/10982
image 11/10982
image 12/10982
image 13/10982
image 14/10982
image 15/10982
image 16/10982
image 17/10982
image 18/10982
image 19/10982
image 20/10982
image 21/10982
image 22/10982
image 23/10982
image 24/10982
image 25/10982
image 26/10982
image 27/10982
image 28/10982
image 29/10982
image 30/10982
image 31/10982
image 32/10982
image 33/10982
image 34/10982
image 35/10982
image 36/10982
image 37/10982
image 38/10982
image 39/10982
image 40/10982
image 41/10982
image 42/10982
image 43/10982
image 44/10982
image 45/10982
image 46/10982
image 47/10982
image 48/10982
image 49/10982
image 50/10982
image 51/10982
image 52/10982
image 53/10982
image 54/10982
image 55/10982
image 56/10982
image 57/10982
image 58/10982
image 59/10982
image 60/10982
image 61/10982
image 62/10982
image 63/10982
image 64/10982
image 65/10982
image 66/10982
image 67/10982
imag

In [53]:
submission = pd.DataFrame()

In [54]:
submission['image_name'] = test_df['image_name']

In [55]:
submission['target'] = predictions

In [56]:
submission.head()

,image_name,target
0,ISIC_0052060,1.505930e-06
1,ISIC_0052349,5.440273e-07
2,ISIC_0058510,2.407922e-07
3,ISIC_0073313,2.277733e-06
4,ISIC_0073502,7.495397e-03


In [57]:
submission.to_csv('submission12.csv', index=False)

## Fine-tuning InceptionV3

In [58]:
for i, layer in enumerate(inceptionv3_model.layers):
   print(i, layer.name)

0 input_1
1 conv2d
2 batch_normalization
3 activation
4 conv2d_1
5 batch_normalization_1
6 activation_1
7 conv2d_2
8 batch_normalization_2
9 activation_2
10 max_pooling2d
11 conv2d_3
12 batch_normalization_3
13 activation_3
14 conv2d_4
15 batch_normalization_4
16 activation_4
17 max_pooling2d_1
18 conv2d_8
19 batch_normalization_8
20 activation_8
21 conv2d_6
22 conv2d_9
23 batch_normalization_6
24 batch_normalization_9
25 activation_6
26 activation_9
27 average_pooling2d
28 conv2d_5
29 conv2d_7
30 conv2d_10
31 conv2d_11
32 batch_normalization_5
33 batch_normalization_7
34 batch_normalization_10
35 batch_normalization_11
36 activation_5
37 activation_7
38 activation_10
39 activation_11
40 mixed0
41 conv2d_15
42 batch_normalization_15
43 activation_15
44 conv2d_13
45 conv2d_16
46 batch_normalization_13
47 batch_normalization_16
48 activation_13
49 activation_16
50 average_pooling2d_1
51 conv2d_12
52 conv2d_14
53 conv2d_17
54 conv2d_18
55 batch_normalization_12
56 batch_normalization_14
5

In [59]:
for layer in model.layers[:249]:
    layer.trainable = False
for layer in model.layers[249:]:
    layer.trainable = True

In [60]:
from tensorflow.keras.optimizers import Adam

In [61]:
model.compile(optimizer=Adam(lr=0.0001),
             loss='binary_crossentropy',
             metrics=['AUC'])

In [70]:
history2 = model.fit(
    train_generator,
    epochs = 40,
    callbacks=[TqdmCallback(verbose=2), cp_callback],
    verbose=0
)

  ...
    to  
  ['...']



Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model to training_2/cp-0001.ckpt

Epoch 00001: saving model t


Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model to training_2/cp-0002.ckpt

Epoch 00002: saving model t


Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model to training_2/cp-0003.ckpt

Epoch 00003: saving model t


Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model to training_2/cp-0004.ckpt

Epoch 00004: saving model t

...


Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model to training_2/cp-0036.ckpt

Epoch 00036: saving model t


Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model to training_2/cp-0037.ckpt

Epoch 00037: saving model t


Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model to training_2/cp-0038.ckpt

Epoch 00038: saving model t


Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model to training_2/cp-0039.ckpt

Epoch 00039: saving model t


Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00040: saving model t

In [71]:
submission5 = pd.DataFrame()

In [72]:
test_df = pd.read_csv('test.csv')

In [73]:
i = 0
predictions = []
for image_name in test_df['image_name']:
    i += 1
    print(f"image {i}/{test_df.shape[0]}")
    filename = image_name+".png"
    img = cv2.imread(r'D:\uni\masters\ML\project\my_code\archive\test\\'+filename)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    x = tf.keras.applications.inception_v3.preprocess_input(img)
    x = np.reshape(x, (1, 224, 224, 3))
    target = model.predict(x)
    predictions.append(target[0][0])

image 1/10982
image 2/10982
image 3/10982
image 4/10982
image 5/10982
image 6/10982
image 7/10982
image 8/10982
image 9/10982
image 10/10982
image 11/10982
image 12/10982
image 13/10982
image 14/10982
image 15/10982
image 16/10982
image 17/10982
image 18/10982
image 19/10982
image 20/10982
image 21/10982
image 22/10982
image 23/10982
image 24/10982
image 25/10982
image 26/10982
image 27/10982
image 28/10982
image 29/10982
image 30/10982
image 31/10982
image 32/10982
image 33/10982
image 34/10982
image 35/10982
image 36/10982
image 37/10982
image 38/10982
image 39/10982
image 40/10982
image 41/10982
image 42/10982
image 43/10982
image 44/10982
image 45/10982
image 46/10982
image 47/10982
image 48/10982
image 49/10982
image 50/10982
image 51/10982
image 52/10982
image 53/10982
image 54/10982
image 55/10982
image 56/10982
image 57/10982
image 58/10982
image 59/10982
image 60/10982
image 61/10982
image 62/10982
image 63/10982
image 64/10982
image 65/10982
image 66/10982
image 67/10982
imag

In [74]:
submission5['image_name'] = test_df['image_name']

In [75]:
submission5['target'] = predictions

In [76]:
submission5.head()

,image_name,target
0,ISIC_0052060,0.000000e+00
1,ISIC_0052349,0.000000e+00
2,ISIC_0058510,0.000000e+00
3,ISIC_0073313,0.000000e+00
4,ISIC_0073502,1.597684e-23


In [78]:
submission5.to_csv('submission_miracle_continue.csv', index=False)